# Klasyfikacja cyfr MNIST za pomocą sieci MLP (TensorFlow Keras)

&#x20;

## Opis problemu i zbioru danych MNIST

Zadaniem jest zbudowanie modelu sieci neuronowej typu **Multi-Layer Perceptron (MLP)** w celu klasyfikacji obrazów przedstawiających odręczne cyfry 0–9 ze zbioru **MNIST**. MNIST to klasyczny zbiór danych w wizji komputerowej, zawierający **70 000 obrazów** czarno-białych (grayscale) ręcznie pisanych cyfr od 0 do 9. Standardowy podział to **60 000 obrazów treningowych i 10 000 testowyc** Każdy obraz ma rozdzielczość **28×28 pikseli**, a piksele zapisane są w skali szarości (wartości od 0 do 255, gdzie 0 oznacza kolor czarny, a 255 – biały). Cyfry w obrazach zostały wycentrowane i znormalizowane rozmiarem w obrębie tych 28×28 pikseli.

![](https://upload.wikimedia.org/wikipedia/commons/b/b1/MNIST_dataset_example.png)


Celem modelu będzie **przydzielenie etykiety 0–9 dla dowolnego obrazu cyfry**. Zbudujemy zatem sieć MLP, która przyjmie 28×28 pikseli jako wejście, przekształci je w ciąg cech, a następnie poprzez kilka warstw w pełni połączonych (fully-connected) wyznaczy prawdopodobieństwa przynależności do każdej z 10 klas (cyfr). Model zostanie **wytrenowany** na danych treningowych, a następnie oceniony na danych testowych.


**Multi-Layer Perceptron (MLP)** to rodzaj sieci neuronowej składającej się z wielu warstw neuronów (jeden lub więcej tzw. warstw ukrytych między wejściem a wyjściem) Każdy neuron w takiej sieci stosuje nieliniową funkcję aktywacji, co pozwala sieci uczyć się złożonych, nieliniowych wzorców w danych[.](https://www.datacamp.com/tutorial/multilayer-perceptrons-in-machine-learning#:~:text=A%20multi,because%20they%20can%20learn%20nonlinear) MLP jest siecią **feed-forward**, co oznacza, że informacje przepływają tylko w jedną stronę – od warstwy wejściowej, przez warstwy ukryte, do wyjściowej. Wszystkie neurony w kolejnych warstwach są w pełni połączone z neuronami z warstwy poprzedniej. W warstwie wyjściowej MLP dla klasyfikacji znajduje się zazwyczaj tyle neuronów, ile jest klas – w naszym przypadku 10 neuronów (odpowiadających cyfrom 0–9) z funkcją aktywacji softmax, która zamieni wyjścia na **rozkład prawdopodobieństwa** na klasy.


## Eksploracja i przygotowanie danych MNIST

Na początek zaimportujemy potrzebne biblioteki i wczytamy zbiór danych MNIST. Można skorzystać z modułu `tensorflow.keras.datasets` lub innej biblioteki udostępniającej MNIST. Następnie zbadamy podstawowe własności danych.



Sprawdźmy teraz kilka przykładowych obrazów i ich etykiety, aby upewnić się, że dane są wczytane poprawnie i zrozumieć, jak wyglądają cyfry:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Wczytanie zbioru MNIST z Keras
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print("Rozmiar zbioru treningowego:", X_train.shape, " Etykiety:", y_train.shape)
print("Rozmiar zbioru testowego:", X_test.shape, "   Etykiety:", y_test.shape)




Teraz przeprowadzimy wstępne **przetwarzanie danych**. W przypadku sieci neuronowych powszechną praktyką jest **normalizacja** danych wejściowych. Piksele mają wartości 0–255, więc znormalizujemy je dzieląc przez 255, aby zawierały się w zakresie \[0, 1]. Taka skalowanie często ułatwia i przyspiesza trenowanie modelu, ponieważ stabilizuje wartości wejść i gradientów.



Ponadto upewnimy się, że etykiety są we właściwym formacie. W Keras do klasyfikacji wieloklasowej można użyć etykiet w formie **sparso-kategorycznej** (czyli po prostu liczby 0–9, wraz z użyciem `sparse_categorical_crossentropy` jako funkcji straty) lub skonwertować je do postaci **one-hot** (wektorów binarnych długości 10) i użyć `categorical_crossentropy`. My skorzystamy z pierwszej opcji – etykiety 0–9 w postaci integerów, co jest prostsze.

Na koniec, dla potrzeby walidacji modelu w trakcie treningu, wydzielimy z danych treningowych mniejszy zbiór **walidacyjny** (np. 10% treningu). Posłuży on do oceny modelu w trakcie uczenia (do np. early stopping).

In [ ]:
# Przetwarzanie danych
# Normalizacja pikseli do zakresu [0, 1]




Przeuczenie (*overfitting*) to sytuacja, gdy model uczy się zbyt szczegółowo danych treningowych i traci uogólnienie – objawia się to wysoką skutecznością na treningu, ale słabą na danych testowych. W dalszej części zastosujemy różne techniki regularyzacji i callbacki, aby zminimalizować przeuczenie.

## Budowa modelu MLP w TensorFlow Keras

Czas zdefiniować architekturę modelu **MLP** do klasyfikacji cyfr. Wykorzystamy wysokopoziomowe API Keras (część biblioteki TensorFlow) do stworzenia modelu sekwencyjnego (`Sequential`). Sieć MLP przyjmie wejście 28×28 pikseli, które najpierw **spłaszczymy** do wektora 784 cech (28\*28) za pomocą warstwy `Flatten`. Następnie dodamy **warstwy gęste (Dense)**, czyli w pełni połączone.

Zaczniemy od jednej lub dwóch warstw ukrytych z określoną liczbą neuronów i funkcją aktywacji **ReLU** (rectified linear unit), która jest standardową nieliniowością używaną w warstwach ukrytych. Na końcu dodamy warstwę wyjściową Dense z 10 neuronami i aktywacją **softmax** (aby model zwracał prawdopodobieństwa dla każdej z 10 klas, sumujące się do 1).



Zwróćmy uwagę na kilka technik, które chcemy uwzględnić w modelu, aby poprawić jego uogólnienie:

* **Dropout** – warstwa regularyzacyjna, która podczas treningu *losowo wyłącza pewien procent neuronów* warstwy poprzednie Dropout zapobiega zbytniej współzależności neuronów i wymusza, by sieć była bardziej odporna (każdy neuron nie może polegać wyłącznie na kilku konkretnych połączeniach, bo te mogą zostać wyzerowane). W praktyce dropout znacząco zmniejsza przeuczenie modelu. Warstwę Dropout zazwyczaj stosuje się **po** warstwie Dense (lub konwolucyjnej) podczas treningu, a wyłączana jest podczas testowania/predykcji.
* **Regularyzacja L2** – inaczej nazywana *weight decay* lub grzbietową (ridge) – polega na dodaniu do funkcji kosztu kary równej sumie kwadratów wag sieci (pomnożonej przez niewielki współczynnik). Dzięki temu sieć jest zachęcana do utrzymywania wag na możliwie małych wartościach, co zapobiega nadmiernemu dopasowaniu do danych treningowych (zbyt duże wagi mogą wskazywać, że model stara się mocno dopasować do szumu/treningu). W Keras możemy dodać regularyzator L2 do każdej warstwy Dense poprzez parametr `kernel_regularizer`.


In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

# model = Sequential([
    
# ])



*Podpowiedź:* Aby dodać regularyzację L2 do warstwy Dense, można użyć parametru `kernel_regularizer=tf.keras.regularizers.L2(lambda)`. Na przykład, `Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001))` doda karę L2 z współczynnikiem 0.001 dla wag tej warstwy. **Zadanie (opcjonalne):** Spróbuj dodać regularyzator L2 do warstw ukrytych i zaobserwuj wpływ na trenowanie modelu (większe lambda -> silniejsza regularyzacja).

Po uzupełnieniu powyższych zadań, możemy sprawdzić strukturę modelu:



Polecenie `model.summary()` wypisze podsumowanie architektury modelu, listując warstwy, kształty wyjść z warstw oraz liczbę parametrów (wag). 



Jeśli wszystko wygląda poprawnie, przejdziemy do ustawienia procesu trenowania.

## Kompilacja modelu i przygotowanie procesu trenowania

Zanim rozpoczniemy trenowanie, musimy **skompilować** model, czyli zdefiniować funkcję straty, optymalizator i metryki, według których model będzie się uczyć i które będą raportowane. Dla zadania klasyfikacji 10-klasowej użyjemy:

* **Funkcja straty:** *Sparse Categorical Crossentropy* – powszechnie stosowana do wieloklasowej klasyfikacji, gdy etykiety są w formie integerów (0–9). Jest to entropia krzyżowa między rozkładem prawdziwym (one-hot zakodowana etykieta) a rozkładem predykcji modelu (softmax).
* **Optymalizator:** np. *Adam* – często wybierany ze względu na skuteczność i szybkość konwergencji.
* **Metryka:** *accuracy* (dokładność klasyfikacji), żeby śledzić odsetek poprawnych klasyfikacji.

In [ ]:
# model.compile()


Teraz przygotujemy **callbacki**, czyli pomocnicze funkcje wywoływane w trakcie trenowania. W szczególności skorzystamy z:

* **EarlyStopping:** czyli wczesnego zatrzymania treningu. Będzie on monitorował stratę walidacyjną (lub dokładność walidacyjną) i przerwie trening, jeśli przez pewną liczbę epok nie nastąpi poprawa. Early stopping zapobiega nadmiernemu trenowaniu po osiągnięciu optymalnego punktu – zatrzymuje w momencie, gdy model przestaje się poprawiać na zbiorze walidacyjny. Ustawimy np. `patience=3` (przerwij, jeśli przez 3 kolejne epoki nie ma poprawy) oraz `restore_best_weights=True` (po przerwaniu przywróć najlepsze wagi modelu z okresu treningu).
* **Dynamiczna zmiana learning rate:** uczenie się może przebiegać lepiej, jeśli **współczynnik uczenia** (learning rate) będzie zmniejszany w trakcie treningu. Możemy to zrobić na dwa sposoby:

  * **LearningRateScheduler:** ustalamy z góry harmonogram zmiany learning rate zależnie od numeru epoki (np. co pewną liczbę epok zmniejszaj lr)
  * **ReduceLROnPlateau:** automatycznie zmniejsza learning rate, gdy miara (np. strata walidacyjna) przestaje się poprawiać. Np. możemy zmniejszyć lr 10-krotnie, jeśli przez określoną liczbę epok nie widać poprawy.


W naszym przypadku skorzystamy z `ReduceLROnPlateau`, aby uprościć konfigurację – będzie on monitorował wartość błędu walidacyjnego (`val_loss`) i jeśli nie będzie poprawy np. przez 2 epoki, to zmniejszy lr o czynnik 0.5. Ustawimy również minimalny lr, poniżej którego nie będzie zmniejszać, np. 1e-5.


Przygotujmy callbacki:


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau






W powyższym kodzie zdefiniowaliśmy:

* `early_stop`: monitoruje `val_loss`, ma `patience=3`, co oznacza że jeśli przez 3 kolejne epoki strata walidacyjna nie spadnie (czyli model nie poprawia się na walidacji), trening zostanie zakończony. Dzięki `restore_best_weights=True` model po treningu będzie miał wagi z epoki o najniższym `val_loss`.
* `reduce_lr`: monitoruje `val_loss` i jeżeli przez 2 epoki nie nastąpi jego poprawa, to zmniejszy learning rate (`factor=0.5` oznacza, że nowy lr = 0.5 \* stary lr). `min_lr=1e-5` zabezpiecza przed zbyt mocnym zmniejszeniem lr.




**Zadanie:** Dodaj powyższe callbacki do listy callbacków i uwzględnij je podczas trenowania modelu. (Poniżej w wywołaniu `model.fit` trzeba przekazać argument `callbacks=callbacks_list`).

*Podpowiedź:* W wywołaniu `model.fit(...)` użyj parametru `callbacks=callbacks_list`. Upewnij się, że trenujesz model z użyciem zbioru walidacyjnego (`validation_data=(X_val, y_val)`).



## Trenowanie modelu

Mając skompilowany model i przygotowane callbacki, możemy rozpocząć trenowanie. Ustalmy liczbę epok (np. początkowo 20) i batch size (np. 32). Dzięki early stopping i reduce LR realna liczba epok może być mniejsza – trening przerwie się wcześniej, jeśli model przestanie poprawiać wyniki.




Podczas treningu w każdej epoce Keras wyświetli stratę i accuracy dla treningu oraz walidacji. Dzięki callbackom:

* EarlyStopping przerwie uczenie, gdy dalsze epoki nie przynoszą poprawy (z komunikatu w konsoli powinniśmy zobaczyć, że trening zatrzymał się np. na wcześniejszej epoce niż 20).
* ReduceLROnPlateau będzie zmniejszać learning rate – Keras zazwyczaj wypisuje komunikat o zmniejszeniu lr, np. "Epoch X: ReduceLROnPlateau reducing learning rate to ...".



Po zakończeniu treningu (czy to po 20 epokach, czy wcześniej z powodu early stopping) mamy wytrenowany model. Wykorzystując atrybut `history.history`, możemy przeanalizować przebieg uczenia:

Wykres strat treningowej i walidacyjnej pozwoli ocenić, czy model się przeucza (jeśli krzywa walidacyjna zaczyna rosnąć, a treningowa nadal spada, to znak przeuczenia). Wykres dokładności pokaże, do jakiego poziomu doszliśmy. Dobrze wytrenowany MLP na MNIST powinien osiągać **dokładność powyżej 95%** na zbiorze walidacyjnym, a nawet zbliżoną do 98% na testowym



## Ewaluacja modelu na zbiorze testowym

Po treningu (i ewentualnym dostrojeniu modelu na walidacji) wykonujemy **ostateczną ocenę** na zbiorze testowym, który jest dla modelu zupełnie nowy. Użyjemy do tego metody `model.evaluate` oraz dodatkowo wyznaczymy **macierz pomyłek** (confusion matrix) i przeanalizujemy wyniki dla poszczególnych klas.



Dokładność (`accuracy`) na zbiorze testowym pokazuje ogólny odsetek poprawnie sklasyfikowanych przykładów. Jednak warto spojrzeć głębiej – które cyfry model rozpoznaje najgorzej, czy myli pewne cyfry ze sobą? Do tego służy **macierz pomyłek**. Macierz pomyłek to tablica \$10 \times 10\$, gdzie na przekątnej znajdują się liczności poprawnych predykcji dla każdej klasy, a poza przekątną – pomyłki (np. w wierszu 5, kolumnie 3 byłaby liczba przypadków, gdzie prawdziwą klasą była 5, a model błędnie zaklasyfikował jako 3)

Wyznaczymy macierz pomyłek dla naszych przewidywań na zbiorze testowym i wyświetlimy ją graficznie:




Na koniec, możemy przetestować model **na kilku przykładowych obrazach** ze zbioru testowego, aby zobaczyć, jak wygląda jego klasyfikacja "w akcji". Weźmy kilka losowych obrazków testowych, wyświetlmy je obok siebie z przewidzianą etykietą oraz prawdziwą etykietą:


## Podsumowanie

W tym notatniku przeprowadziliśmy proces treningu sieci MLP do rozpoznawania odręcznych cyfr z MNIST. Omówione zostały kolejne kroki: od eksploracji i przygotowania danych, poprzez zbudowanie modelu, aż do jego trenowania z użyciem różnych technik poprawiających uogólnienie:

* **Dropout** – losowe wyłączanie neuronów podczas treningu, aby zapobiegać przeuczeniu
* **Regularyzacja L2** – karanie dużych wag w modelu, co również ogranicza przeuczenie
* **Dynamiczna zmiana szybkości uczenia** – zmniejszanie learning rate gdy trening się stabilizuje, co umożliwia lepsze dopasowanie modelu (np. `ReduceLROnPlateau`)
* **Early Stopping** – wczesne zatrzymanie treningu, gdy model przestaje się poprawiać na zbiorze walidacyjnym, co chroni przed nadmiernym trenowaniem

.

**Zadania do samodzielnego wykonania:** Jeśli chcesz dalej poeksperymentować:

* Spróbuj zmodyfikować architekturę modelu: zwiększ lub zmniejsz liczbę neuronów w warstwach ukrytych, dodaj trzecią warstwę ukrytą itp. Zaobserwuj, jak to wpływa na wynik.
* Przetestuj różne wartości parametru dropout (np. 0.5) lub współczynnika regularyzacji L2. Czy zbyt duży dropout lub zbyt silna regularyzacja pogorszą wyniki (niedouczenie)?
* Wypróbuj **LearningRateScheduler** zamiast ReduceLROnPlateau – np. ustaw harmonogram, który co kilka epok zmniejsza learning rate o stały czynnik.
* Wygeneruj **raport klasyfikacji** (precision, recall dla każdej klasy) używając `classification_report` z scikit-learn, aby zobaczyć dokładne statystyki dla każdej cyfry.
* Porównaj wyniki MLP z bardziej zaawansowanymi modelami, np. siecią konwolucyjną (CNN). Klasyczny LeNet-5 lub prosta CNN na tych samych danych MNIST często przekracza 99% dokładności – możesz spróbować to osiągnąć jako dodatkowe wyzwanie.
